In [1]:
mode="BTW_BFB_FGDA"
# mode="BTW_BFB_FGDA"
# mode="All_TWFB_RCT_FGDA"
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from mne.decoding import CSP
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace,FGDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn import preprocessing
from sklearn.svm import SVC
import pyriemann
from pyriemann.classification import SVC as rSVM
from pyriemann.classification import MDM,FgMDM
import numpy as np
import moabb,sys,os,pickle,itertools
from moabb.datasets import BNCI2014001
from moabb.evaluations import CrossSessionEvaluation,WithinSessionEvaluation
from moabb.paradigms import FilterBankMotorImagery,FilterBankLeftRightImagery, LeftRightImagery,MotorImagery
from moabb.pipelines.utils import FilterBank
from pyriemann.estimation import Covariances


from san_utils import * 
from san_utils_rpa import *
moabb.set_log_level("info")

############### load .pkl file #############3
def loadfilters(direc,subject,n_filters):
    with open(direc+'/sub_filters.pkl', 'rb') as file:
        sub_filters=pickle.load(file)
    with open(direc+'/timewin.pkl', 'rb') as file:
        timewin=pickle.load( file)
    bestfilters=sub_filters[subject][n_filters]
    bestTW=timewin[subject]
   
    return bestfilters,bestTW

# saving the results
def save_csvfile(results,sfile_name):
    if(os.path.isfile('./'+sfile_name) ==True):
        print("exists")
        results.to_csv(sfile_name, mode="a", index=False, header=False)
    else:
        print("No.. so creating")
        results.to_csv(sfile_name,  index=False, header=True) 
    return



In [2]:
pipelines = {}
fb1 = FilterBank(make_pipeline(CSP(nfilter=4)))
fb0 = FilterBank(make_pipeline(FGDA(),CSP(nfilter=4)))
fb2 = FilterBank(make_pipeline(TangentSpace()))
fb3 = FilterBank(make_pipeline(FGDA(),TangentSpace()))

pipelines["FB_CSP_LDA"] = make_pipeline(fb1, LDA())
pipelines["FB_FGDA_CSP_LDA"] = make_pipeline(fb0, LDA())
pipelines["FB_CSP_SVM"] = make_pipeline(fb1, SVC(kernel="linear"))
pipelines["FB_FGDA_CSP_SVM"] = make_pipeline(fb0, SVC(kernel="linear"))

pipelines["FB_TS_LDA"] = make_pipeline(fb2, LDA())
pipelines["FB_TS_SVM"] = make_pipeline(fb2, SVC(kernel="linear"))
pipelines["FB_FGDA_TS_LDA"] = make_pipeline(fb3, LDA())
pipelines["FB_FGDA_TS_SVM"] = make_pipeline(fb3, SVC(kernel="linear"))

# pipelines["MDM"] = make_pipeline(Covariances("oas"), MDM(metric="riemann"))
# pipelines["FgMDM"] = make_pipeline(Covariances("oas"), FgMDM(metric="riemann"))

In [3]:
def FB_to_cov_rSVM(X):
    x_sum_dist=[]
    for i in range(X.shape[1]):
        cov=X[:,i,:]
        cov_sum=np.sum(cov, axis = 0)
#         print("covsum:",cov_sum.shape)
        x_sum_dist.append(cov_sum) 
    x_sum_dist=np.array(x_sum_dist)
    return x_sum_dist

def FB_rSVM(X_train,y_train,X_test,y_test):
    clf=rSVM(probability=True)
#     print(X_train.shape,y_train.shape,np.unique(y_train))
    yps=[]
    for i in range(X_train.shape[-1]):
        xt=X_train[:,:,:,i];xe=X_test[:,:,:,i]
        clf.fit(xt,y_train); yp=clf.predict_proba(xe)
        yps.append(yp)
    yps=np.array(yps)
#     print(yps.shape)
    yprob=FB_to_cov_rSVM(yps)
    y_pred=np.argmax(yprob, axis=1)
    acc=accuracy_score(y_test,y_pred)
    return acc, y_pred


In [5]:
dataset = BNCI2014001()
dataset_name="BNCI2014001"
subjects=[1,2,3,4,5,6,7,8,9]
fmin = 4;fmax = 40; sfreq=250; n_class=4
nbands_Sel=[7]#np.arange(1,18)
nbs=["2468","4","24","246"]
nbs=["4"]

flg=0

for nb in nbs:
    xt={};yt={};xe={};ye={}
    fpath=dataset_name+"/tw_fb"+nb
    sfile_name = fpath+"/Ablation_accResults_CrossSess_"+mode+"_dscoredm_bands"+nb+".csv"
    #filtername=fpath+"/filters.pkl"
    df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc',
    'recall','precision','f1','kappa','sensitivity','specificity'])
    for sub in subjects:
        for nband in nbands_Sel:
            print("subject:",sub," || nbands:",nband)
            bestfilters,bestTW=loadfilters(direc=fpath,subject=sub,n_filters=nband) #from .pkl file
            paradigm = FilterBankMotorImagery(filters=bestfilters,n_classes=n_class)
            X, y, meta = paradigm.get_data(dataset=dataset, subjects=[sub])
            if nband==1:
                X=np.resize(X,(X.shape[0],X.shape[1],X.shape[2],1))
            nchan=X.shape[1]
            X=X[:,:,int(sfreq*bestTW[0]):int(sfreq*bestTW[1])]
            select_indices = list(np.where(meta["session"] == "session_T")[0])
            X0=X[select_indices,:];y0=y[select_indices]
            select_indices = list(np.where(meta["session"] == "session_E")[0])
            X1=X[select_indices,:];y1=y[select_indices]

            if flg==0:
                le,le_label_map=label_map_fn(y0) 
                flg=1

            y0 = le.transform(y0);y1 = le.transform(y1)

#             X_train, X_test, y_train, y_test=RCT_transform_nosplit(X0, X1, y0, y1)
            # X_train, X_test, y_train, y_test=calib_transform(X_train1, X_test1, y_train1, y_test1,target_train_split=10)
            #X_train, X_test, y_train, y_test=RPA_transform(X_train1, X_test1, y_train1, y_test1,target_train_split=10)
            # X_train, X_test, y_train, y_test=RCT_transform(X_train1, X_test1, y_train1, y_test1,target_train_split=10)
            X_train, X_test, y_train, y_test=no_transform_baseline(X0, X1, y0, y1)
            xt[(sub,nband)]=X_train;   yt[(sub,nband)]=y_train
            xe[(sub,nband)]=X_test;    ye[(sub,nband)]=y_test
            print(X_train.shape,np.unique(y_train))
            for pipe in pipelines.keys():
                #print(pipe)
                pipelines[pipe].fit(X_train, y_train)
                y_pred=pipelines[pipe].predict(X_test)
                acc,recall,precision,f1,kappa,sens,spec=all_metrics(y_test,y_pred)
                print(pipe,":",acc)
                #append row to the dataframe
                df.loc[len(df)] = [dataset_name, "session_E", nchan, sub, nband, pipe, acc,recall,precision,f1,kappa,sens,spec]
            #### FB_MDM ###
            pipe="FB_MDM"
            fb4 = FilterBank(make_pipeline(MDM()), flatten=False)
            a,y_pred=FB_MDM(X_train,y_train,X_test,y_test,fb4)
            acc,recall,precision,f1,kappa,sens,spec=all_metrics(y_test,y_pred)
            print(pipe,":",acc)
            df.loc[len(df)] = [dataset_name, "session_E", nchan, sub, nband, pipe, acc,recall,precision,f1,kappa,sens,spec]
            #### FB_FgMDM ###
            pipe="FB_FgMDM"
            fb5 = FilterBank(make_pipeline(FgMDM()), flatten=False)
            a,y_pred=FB_FgMDM(X_train,y_train,X_test,y_test,fb5)
            acc,recall,precision,f1,kappa,sens,spec=all_metrics(y_test,y_pred)
            print(pipe,":",acc)
            df.loc[len(df)] = [dataset_name, "session_E", nchan, sub, nband, pipe, acc,recall,precision,f1,kappa,sens,spec]
            #### FGDA_rSVM #####
            pipe="FGDA_rSVM"
            fb6 = FilterBank(make_pipeline(FGDA()),flatten=False)
            X_train1=fb6.fit_transform(X_train,y_train);X_test1=fb6.transform(X_test)
            a,y_pred=FB_rSVM(X_train1,y_train,X_test1,y_test)
            acc,recall,precision,f1,kappa,sens,spec=all_metrics(y_test,y_pred)
            print(pipe,":",acc)
            df.loc[len(df)] = [dataset_name, "session_E", nchan, sub, nband, pipe, acc,recall,precision,f1,kappa,sens,spec]
            #### rSVM #####
            pipe="rSVM"
            a,y_pred=FB_rSVM(X_train,y_train,X_test,y_test)
            acc,recall,precision,f1,kappa,sens,spec=all_metrics(y_test,y_pred)
            print(pipe,":",acc)
            df.loc[len(df)] = [dataset_name, "session_E", nchan, sub, nband, pipe, acc,recall,precision,f1,kappa,sens,spec]
            

            
            
save_csvfile(df,sfile_name)


2022-12-28 11:42:26,381 WARNING MainThread moabb.paradigms.motor_imagery Choosing from all possible events


subject: 1  || nbands: 7
(288, 22, 22, 7) [0 1 2 3]
FB_CSP_LDA : 0.8472222222222222


ValueError: expected square "a" matrix

In [ ]:
print(X_train.shape,np.unique(y_train))
fb6.fit(X_train,y_train)
x_t=fb6.transform(X_train)
print(x_t.shape)
x_tt=np.reshape(x_t,(x_t.shape[0]*x_t.shape[3],x_t.shape[1],x_t.shape[2]))
print(x_tt.shape)

In [5]:
### save X_train, X_test, y_train, y_test in .pkl file for 3Dcnn
def save_pkl(xt,xe,yt,ye,le_map,direc):
    with open(direc+'/xt_cnn_all.pkl', 'wb') as file:
        pickle.dump(xt, file)
    with open(direc+'/xe_cnn_all.pkl', 'wb') as file:
        pickle.dump(xe, file)
    with open(direc+'/yt_cnn_all.pkl', 'wb') as file:
        pickle.dump(yt, file)
    with open(direc+'/ye_cnn_all.pkl', 'wb') as file:
        pickle.dump(ye, file) 
    with open(direc+'/le_map_cnn_all.pkl', 'wb') as file:
        pickle.dump(le_map, file) 

save_pkl(xt,xe,yt,ye,le_label_map,fpath)

In [8]:
# sfile_name='BNCI2014001/tw_fb4/plot_allbands_1_17.csv'
# save_csvfile(df,sfile_name)

No.. so creating
